# Extracting data from URLs
In this notebook 50K urls data has been extracted and have been saved in a text or pdf while, that data can be used to clean and gets insights from it. Multithreading was used in this notebook as to save the time.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from os import getcwd, path
from requests.exceptions import HTTPError
import pickle
from time import sleep
import functools
import requests
from func_timeout import func_timeout, FunctionTimedOut
from bs4 import BeautifulSoup as soup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
df = pd.read_csv('StatementSummaries2020.csv')

C:\Users\Divyanshu\AppData\Local\Temp\ipykernel_10084\1777518964.py:1: DtypeWarning: Columns (16,35,56,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('StatementSummaries2020.csv')


In [3]:
df.columns

Index(['StatementYear', 'OrganisationName', 'Address', 'SectorType',
       'CompanyNumber', 'LastUpdated', 'GroupSubmission', 'ParentName',
       'StatementURL', 'EmailAddressNoURL', 'StatementSummaryURL',
       'StatementStartDate', 'StatementEndDate', 'ApprovingPerson',
       'DateApproved', 'StatementIncludesOrgStructure', 'NoOrgStructureReason',
       'StatementIncludesPolicies', 'NoPoliciesReason',
       'StatementIncludesRisksAssessment', 'NoRiskAssessmentReason',
       'StatementIncludesDueDiligence', 'NoDueDiligenceReason',
       'StatementIncludesTraining', 'NoTrainingReason',
       'StatementIncludesGoals', 'NoGoalsReason', 'OrganisationSectors',
       'OtherOrganisationSector', 'Turnover', 'YearsProducingStatements',
       'Policies', 'OtherPolicies', 'NoPolicies', 'Training', 'OtherTraining',
       'NoTraining', 'WorkingConditionsEngagement', 'NoEngagement',
       'SocialAudits', 'NoSocialAudits', 'GrievanceMechanisms',
       'NoGrievanceMechanisms', 'OtherMon

In [4]:
len(np.unique(df['OrganisationName']))

8046

In [5]:
len(df)

8243

### There were few Nan values in Statement URL and hence dropped those rows

In [6]:
df = df[df['StatementURL'].notna()]

In [7]:
df

,StatementYear,OrganisationName,Address,SectorType,CompanyNumber,LastUpdated,GroupSubmission,ParentName,StatementURL,EmailAddressNoURL,...,Risk3OtherGroupAffected,Risk3Location,Risk3Mitigation,ILOIndicatorsInStatement,NoILOIndicatorsInStatement,ILOIndicatorsActions,ILOIndicatorsNoActions,DemonstrateProgress,PDFDate,PDFURL
0,2020,"""Edelweiss Air Ltd.""",Flughafen Kloten\r\nZürich\r\nSwitzerland\r\n8058,Private,NaN,25/06/2021,No,NaN,https://www.flyedelweiss.com/DE/about-edelweis...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,.BIG YELLOW SELF STORAGE COMPANY LIMITED,2 The Deans Bridge Road\r\nBagshot\r\nSurrey\r...,Private,05166173,07/04/2021,Yes,BIG YELLOW GROUP PLC,https://corporate.bigyellow.co.uk/investors/co...,NaN,...,a range of groups,NaN,We will be updating our 2020/21 Statement with...,NaN,Yes,NaN,NaN,In 2018 we conducted a detailed spend-based ri...,NaN,NaN
2,2020,0404 INVESTMENTS LIMITED,"Pendle Court, Unit 4 Shuttleworth Mead Busines...",Private,05106531,29/04/2021,Yes,WHAT MORE UK LIMITED,https://www.whatmoreuk.com/modern-slavery-huma...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,0404 INVESTMENTS LIMITED,"Pendle Court, Unit 4 Shuttleworth Mead Busines...",Private,05106531,29/04/2021,Yes,NaN,https://www.whatmoreuk.com/modern-slavery-huma...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,1 SMITHFIELD SQUARE LTD,7200 The Quorum\r\nOxford Business Park North\...,Private,08623011,08/04/2021,Yes,GRE CAPITAL LTD,https://greassets.co.uk/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8238,2020,"hp-TECHNIK GmbH Industriepumpen, Förderaggrega...",NaN,Private,NaN,10/06/2021,Yes,Danfoss,https://files.danfoss.com/download/CorporateCo...,NaN,...,NaN,NaN,NaN,Retention of identity documents\r\nExcessive o...,No,Other,No,We have set progress goals for our next statem...,NaN,NaN
8239,2020,mp-montageprofis-GmbH & Co. KG,NaN,Private,NaN,03/09/2021,Yes,BECK & POLLITZER LIMITED,https://beck-pollitzer.com/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8240,2020,mp-montageprofis-Verwaltungs GmbH.,NaN,Private,NaN,03/09/2021,Yes,BECK & POLLITZER LIMITED,https://beck-pollitzer.com/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8241,2020,pitchsport International Ltd,NaN,Private,NaN,28/07/2021,Yes,HOMER NEWCO LIMITED,https://pitchinternational.com/modern-slavery-...,NaN,...,NaN,NaN,none,NaN,NaN,NaN,NaN,we will look to roll-out more comprehensive tr...,NaN,NaN


In [8]:
cols = ['OrganisationName','StatementURL']

In [9]:
df = df[cols]

In [10]:
df.drop_duplicates(inplace = True)

C:\Users\Divyanshu\AppData\Local\Temp\ipykernel_10084\3424306917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace = True)


In [11]:
len(df)

7693

In [12]:
import pickle
PIK = "pickle.dat"

data = ["A", "b", "C", "d"]
with open(PIK, "wb") as f:
    pickle.dump(data, f)
with open(PIK, "rb") as f:
    file = (pickle.load(f))
file

['A', 'b', 'C', 'd']

## Initial one time load for pickle file that will contain whole csv organisation name

In [13]:
Current_path = os.getcwd().replace('\\','/')
Pickle_path = Current_path+'/Picklefiles/'
PDFs_path = Current_path+'/PDFs/'
Text_path = Current_path+'/Texts/'

In [14]:
# pickle functions are created as to monitor which organisations data has been extracted
def write_pickle(directory, pkl_name,data):
    pkl_name = pkl_name+'.dat'
    with open(directory+pkl_name, "wb") as f:
        pickle.dump(data, f)
        
def read_pickle(directory,filename):
    path = directory+filename+'.dat'
    with open(path, "rb") as f:
        file = (pickle.load(f))
        return file

#### Loading whole organisation to pickle file to iterate the list
Passing only the unique values of Organisation Name as data 

In [15]:
write_pickle(Pickle_path,'OrganisationName',list(df['OrganisationName'].drop_duplicates().values))

In [16]:
# For the first time it'll contain whole list of Organisation Name that are in CSV file
# ON - Acroynm for Organisation Name
ON_URLs_to_extract = read_pickle(Pickle_path,'OrganisationName')

In [68]:
def save_pdf(pdf_response, filename):
    """Save text in text file."""
    file_name = filename.replace('"', '')
    try:
        file_obj = open(f"{PDFs_path}//{file_name}.pdf", 'w', encoding='utf-8')
        file_obj.write(pdf_response)
    except TypeError as type_error:
        try:
            file_obj = open(f"{PDFs_path}//{file_name}.pdf", 'wb')
            file_obj.write(pdf_response)
        except Exception as e:
            print(e)
    file_obj.close()
#     print(filename+' PDF created')
    
def save_text(text, filename):
    """Save text in text file."""
    file_name = filename.replace('"', '')
    file_obj = open(f"{Text_path}//{file_name}.txt", 'w', encoding='utf-8')
    file_obj.write(text)
    file_obj.close()
#     print(filename+' text file created')
    
def is_response_html(response):
    if "<html>" in str(response) or "!DOCTYPE html" in str(response) or "!doctype html" in str(response):
        return True
    return False

def is_response_pdf(response):
    if "%PDF" in str(response):
        return True
    return False

def clean_html_response(html_response):
    text = soup(str(html_response)).text
    text = clean_text(text)
    special_char = ["\\", "b'"]
    for char_ in special_char:
        text = text.replace(char_, " ")
    cleaned_text = []
    for word in text.split():
        if len(word) > 1:
            cleaned_text.append(word)
    cleaned_text = " ".join(cleaned_text) 
    return cleaned_text

def clean_text(text):
    """Clean and return input text."""
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    return " ".join(text.split())

In [21]:
# Iterating for each organisation (For some organisation URLs are same, hence considering the organisation name as primary key)
import time
from tqdm import tqdm
from threading import Lock
def extract_texts_pdf(remaining_orgNm,pkl_filename,lock):
    lock = Lock()
    leftover_OrgNm = remaining_orgNm.copy()
    print('Total Organization URLs to be processesd '+ str(len(remaining_orgNm)))
    session = requests.Session()
    for i,OrgNm in tqdm(enumerate(remaining_orgNm)):
#         print(OrgNm)
        try:
            url = df[df['OrganisationName']==OrgNm]['StatementURL'].values[0] #df is the dataframe created from csv
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            response = requests.get(url, timeout=10).content
            if is_response_html(response=response):
#                 print('The file is text '+OrgNm)
                text_response = clean_html_response(html_response=response)
                text_response = clean_html_response(html_response=text_response)
                save_text(text=text_response, filename=OrgNm)
            if is_response_pdf(response):
                print('The file is PDF '+OrgNm)# if pdf response
                save_pdf(pdf_response=response, filename=OrgNm)
            leftover_OrgNm.remove(OrgNm)
            time.sleep(2)
        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}")
            with lock:
                write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    with lock:
        write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    print('Total Organization URLs processed '+ str(len(remaining_orgNm)-len(leftover_OrgNm)))
    print('Remaining number of organization data to extract '+str(len(leftover_OrgNm)))

In [22]:
# Reading the existing URLS to extract
ON_URLs_to_extract = read_pickle(Pickle_path,'OrganisationName')
len(ON_URLs_to_extract)

7645

In [23]:
# Reading the existing URLS to extract
ON_URLs_to_extract_pkl = read_pickle(Pickle_path,'pkl_filename')
len(ON_URLs_to_extract_pkl)

6675

In [109]:
df[df['OrganisationName']=='121333 LIMITED']['StatementURL'].values[0]

'https://www.thepartsalliance.com/about/governance/'

In [93]:
read_pickle(Pickle_path,'pkl_filename')

['121222 HOLDCO LIMITED',
 '121333 LIMITED',
 'ADDO FOOD GROUP LIMITED',
 'AKA PROMOTIONS LIMITED',
 'ALDRIDGE EDUCATION',
 'ALLANDER BIDCO LIMITED',
 'ALLANDER MIDCO LIMITED',
 'ALLANDER TOPCO LIMITED',
 'ANSTY DEVELOPMENT VEHICLE LLP',
 'APPT CORPORATION LIMITED',
 'AVK/SEG (UK) LIMITED']

In [112]:
url = 'https://www.3i.com/media/4884/msa-statement-2020-signed-16102020.pdf'
# response = requests.get(url, timeout=10, allow_redirects=True).content
# print(response)


retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)

response = requests.get(url, timeout=10, allow_redirects=True).content
save_pdf(pdf_response=response, filename = 'Test')

# Multi threading sample code

In [56]:
ON_URLs_to_extract

['121222 HOLDCO LIMITED',
 '121333 LIMITED',
 'ADDO FOOD GROUP LIMITED',
 'AKA PROMOTIONS LIMITED',
 'ALDRIDGE EDUCATION',
 'ALLANDER BIDCO LIMITED',
 'ALLANDER MIDCO LIMITED',
 'ALLANDER TOPCO LIMITED',
 'ANNODATA LIMITED',
 'ANSTY DEVELOPMENT VEHICLE LLP',
 'APPT CORPORATION LIMITED',
 'AVK/SEG (UK) LIMITED',
 'Aker Solutions Ghana Ltd',
 'Aker Solutions Holding AS',
 'Aker Solutions Inc',
 'Aker Solutions India Operations Sdn Bhd',
 'Aker Solutions India Sdn Bhd',
 'Aker Solutions India Services Sdn Bhd',
 'Aker Solutions Korea Co Ltd',
 'Aker Solutions Malaysia Sdn Bhd',
 'Aker Solutions Middle East AS',
 'Aker Solutions Mocambique Ltda',
 'Aker Solutions Nigeria Ltd',
 'Aker Solutions Pty Ltd',
 'Aker Solutions Russia AS',
 'Aker Solutions SAS',
 'Aker Solutions Saudi Arabia Co. Ltd',
 'Aker Solutions Sdn Bhd',
 'Aker Solutions Tanzania Ltd',
 'Aker Solutions USA Corporation',
 'Aker Solutions Umbilical Asia Pacific Sdn Bhd',
 'Aker Solutions de Brazil Ltda',
 'Aker Solutions de M

In [64]:
checkurls = ['121222 HOLDCO LIMITED',
 '121333 LIMITED',
 'ADDO FOOD GROUP LIMITED',
 'AKA PROMOTIONS LIMITED',
 'ALDRIDGE EDUCATION',
 'ALLANDER BIDCO LIMITED',
 'ALLANDER MIDCO LIMITED',
 'ALLANDER TOPCO LIMITED',
 'ANNODATA LIMITED',
 'ANSTY DEVELOPMENT VEHICLE LLP',
 'APPT CORPORATION LIMITED',
 'AVK/SEG (UK) LIMITED',
 'Aker Solutions Ghana Ltd',
 'Aker Solutions Holding AS',
 'Aker Solutions Inc',
 'Aker Solutions India Operations Sdn Bhd',
 'Aker Solutions India Sdn Bhd',
 'Aker Solutions India Services Sdn Bhd',
 'Aker Solutions Korea Co Ltd',
 'Aker Solutions Malaysia Sdn Bhd',
 'Aker Solutions Middle East AS',
 'Aker Solutions Mocambique Ltda',
 'Aker Solutions Nigeria Ltd',
 'Aker Solutions Pty Ltd',
 'Aker Solutions Russia AS',
 'Aker Solutions SAS']

In [24]:
import concurrent.futures
import requests
from threading import Thread
from threading import Lock

lock = Lock()
threads = [Thread(target=extract_texts_pdf, args=(ON_URLs_to_extract,'OrganisationName',lock))]

for thread in threads:
    thread.start()


Total Organization URLs to be processesd 7645


4it [00:14,  3.85s/it]C:\Users\Divyanshu\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
9it [00:35,  5.08s/it]

Unexpected err=ConnectionError(MaxRetryError("HTTPSConnectionPool(host='www.thepartsalliance.com', port=443): Max retries exceeded with url: /about/governance/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B8574F75E0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))")), type(err)=<class 'requests.exceptions.ConnectionError'>


10it [00:43,  5.99s/it]

Unexpected err=ConnectionError(MaxRetryError("HTTPSConnectionPool(host='www.thepartsalliance.com', port=443): Max retries exceeded with url: /about/governance/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B8574F7E20>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))")), type(err)=<class 'requests.exceptions.ConnectionError'>


...

The file is PDF eDreams, S.R.L.


7640it [8:04:38,  2.35s/it]

The file is PDF hp-TECHNIK GmbH Industriepumpen, Förderaggregate und Anlagenbau --- 12%


7645it [8:05:01,  3.81s/it]

Unexpected err=ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='www.waters.com', port=443): Read timed out. (read timeout=10)")), type(err)=<class 'requests.exceptions.ReadTimeout'>
Total Organization URLs processed 7325
Remaining number of organization data to extract 320


In [85]:
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 2, 3)

# Statement 2021

In [25]:
df_2021 = pd.read_csv('StatementSummaries2021.csv')
len(df_2021)

14986

In [26]:
df_2021 = df_2021[df_2021['StatementURL'].notna()]
len(df_2021)

14499

In [27]:
cols = ['OrganisationName','StatementURL']
df_2021 = df_2021[cols]
df_2021.drop_duplicates(inplace = True)

In [28]:
len(df_2021)

14240

In [32]:
df_2021[df_2021['OrganisationName']=='ALUN GRIFFITHS (CONTRACTORS) LIMITED']

,OrganisationName,StatementURL
597,ALUN GRIFFITHS (CONTRACTORS) LIMITED,https://tarmac.com/modern-slavery/
598,ALUN GRIFFITHS (CONTRACTORS) LIMITED,https://www.alungriffiths.co.uk/modern-slavery...
599,ALUN GRIFFITHS (CONTRACTORS) LIMITED,https://www.crh.com/media/3894/crh1734-modern-...


In [34]:
df_2021.drop_duplicates(subset='OrganisationName',inplace=True)

In [35]:
Current_path = os.getcwd().replace('\\','/')
Pickle_path = Current_path+'/Picklefiles/'
PDFs_path = Current_path+'/PDFs2021/'
Text_path = Current_path+'/Texts2021/'

In [66]:
import time
from tqdm import tqdm
from threading import Lock
def extract_texts_pdf2021(remaining_orgNm,pkl_filename,lock):
    lock = Lock()
    leftover_OrgNm = remaining_orgNm.copy()
    print('Total Organization URLs to be processesd '+ str(len(remaining_orgNm)))
    session = requests.Session()
    for i,OrgNm in tqdm(enumerate(remaining_orgNm)):
#         print(OrgNm)
        try:
            url = df_2021[df_2021['OrganisationName']==OrgNm]['StatementURL'].values[0] #df is the dataframe created from csv
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            response = requests.get(url, timeout=10).content
            if is_response_html(response):
                print('The file is text '+OrgNm)
                text_response = clean_html_response(html_response=response)
                text_response = clean_html_response(html_response=text_response)
                save_text(text=text_response, filename=OrgNm)
            if is_response_pdf(response):
                print('The file is PDF '+OrgNm)# if pdf response
                save_pdf(pdf_response=response, filename=OrgNm)
            leftover_OrgNm.remove(OrgNm)
            time.sleep(2)
        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}")
            with lock:
                write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    with lock:
        write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    print('Total Organization URLs processed '+ str(len(remaining_orgNm)-len(leftover_OrgNm)))
    print('Remaining number of organization data to extract '+str(len(leftover_OrgNm)))

In [70]:
# ON_URLs_to_extract2021 = list(df_2021['OrganisationName'].values)

In [72]:
ON_URLs_to_extract2021 = read_pickle(Pickle_path,'OrganisationName2021')
len(ON_URLs_to_extract2021)

3279

In [73]:
import concurrent.futures
import requests
from threading import Thread
from threading import Lock

lock = Lock()
threads = [Thread(target=extract_texts_pdf2021, args=(ON_URLs_to_extract2021,'OrganisationName2021',lock))]

for thread in threads:
    thread.start()

Total Organization URLs to be processesd 3279


1it [00:08,  8.97s/it]

Unexpected err=ConnectionError(MaxRetryError("HTTPSConnectionPool(host='www.thepartsalliance.com', port=443): Max retries exceeded with url: /about/governance/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B806C3B2B0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))")), type(err)=<class 'requests.exceptions.ConnectionError'>


2it [00:17,  8.51s/it]

Unexpected err=ConnectionError(MaxRetryError("HTTPSConnectionPool(host='www.thepartsalliance.com', port=443): Max retries exceeded with url: /about/governance/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B806C39C60>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))")), type(err)=<class 'requests.exceptions.ConnectionError'>


...

The file is text time:matters GmbH


3278it [4:40:33,  3.08s/it]

The file is PDF zooplus AG


3279it [4:40:35,  5.13s/it]


Total Organization URLs processed 2734
Remaining number of organization data to extract 545


# Statement 2022

In [75]:
df_2022 = pd.read_csv('StatementSummaries2022.csv')
len(df_2022)

7832

In [76]:
df_2022 = df_2022[df_2022['StatementURL'].notna()]
len(df_2022)

7664

In [77]:
cols = ['OrganisationName','StatementURL']
df_2022 = df_2022[cols]
df_2022.drop_duplicates(inplace = True)

In [80]:
df_2022.drop_duplicates(subset='OrganisationName',inplace=True)

In [81]:
len(df_2022)

7585

In [82]:
Current_path = os.getcwd().replace('\\','/')
Pickle_path = Current_path+'/Picklefiles/'
PDFs_path = Current_path+'/PDFs2022/'
Text_path = Current_path+'/Texts2022/'

In [84]:
ON_URLs_to_extract2022 = list(df_2022['OrganisationName'].values)

In [88]:
import time
from tqdm import tqdm
from threading import Lock
def extract_texts_pdf2022(remaining_orgNm,pkl_filename,lock):
    lock = Lock()
    leftover_OrgNm = remaining_orgNm.copy()
    print('Total Organization URLs to be processesd '+ str(len(remaining_orgNm)))
    session = requests.Session()
    for i,OrgNm in tqdm(enumerate(remaining_orgNm)):
#         print(OrgNm)
        try:
            url = df_2022[df_2022['OrganisationName']==OrgNm]['StatementURL'].values[0] #df is the dataframe created from csv
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            response = requests.get(url, timeout=10).content
            if is_response_html(response):
                print('The file is text '+OrgNm)
                text_response = clean_html_response(html_response=response)
                text_response = clean_html_response(html_response=text_response)
                save_text(text=text_response, filename=OrgNm)
            if is_response_pdf(response):
                print('The file is PDF '+OrgNm)# if pdf response
                save_pdf(pdf_response=response, filename=OrgNm)
            leftover_OrgNm.remove(OrgNm)
            time.sleep(2)
        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}")
            with lock:
                write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    with lock:
        write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    print('Total Organization URLs processed '+ str(len(remaining_orgNm)-len(leftover_OrgNm)))
    print('Remaining number of organization data to extract '+str(len(leftover_OrgNm)))

In [89]:
import concurrent.futures
import requests
from threading import Thread
from threading import Lock

lock = Lock()
threads = [Thread(target=extract_texts_pdf2022, args=(ON_URLs_to_extract2022,'OrganisationName2022',lock))]

for thread in threads:
    thread.start()

Total Organization URLs to be processesd 7585


0it [00:00, ?it/s]

The file is PDF Syndicate 3624


1it [00:03,  3.37s/it]

The file is PDF Syndicate 6104


...

The file is text setec international


7584it [7:23:25,  4.00s/it]

The file is PDF zooplus SE


7585it [7:23:28,  3.51s/it]


Total Organization URLs processed 7291
Remaining number of organization data to extract 294


# Statement 2023

In [90]:
df_2023 = pd.read_csv('StatementSummaries2023.csv')
len(df_2023)

210

In [91]:
df_2023 = df_2023[df_2023['StatementURL'].notna()]
len(df_2023)

207

In [92]:
cols = ['OrganisationName','StatementURL']
df_2023 = df_2023[cols]
df_2023.drop_duplicates(inplace = True)

In [93]:
df_2023.drop_duplicates(subset='OrganisationName',inplace=True)

In [94]:
len(df_2023)

199

In [95]:
Current_path = os.getcwd().replace('\\','/')
Pickle_path = Current_path+'/Picklefiles/'
PDFs_path = Current_path+'/PDFs2023/'
Text_path = Current_path+'/Texts2023/'

In [96]:
# ON_URLs_to_extract2023 = list(df_2023['OrganisationName'].values)

In [99]:
ON_URLs_to_extract2023 = read_pickle(Pickle_path,'OrganisationName2023')
len(ON_URLs_to_extract2023)

7

In [97]:
import time
from tqdm import tqdm
from threading import Lock
def extract_texts_pdf2023(remaining_orgNm,pkl_filename,lock):
    lock = Lock()
    leftover_OrgNm = remaining_orgNm.copy()
    print('Total Organization URLs to be processesd '+ str(len(remaining_orgNm)))
    session = requests.Session()
    for i,OrgNm in tqdm(enumerate(remaining_orgNm)):
#         print(OrgNm)
        try:
            url = df_2023[df_2023['OrganisationName']==OrgNm]['StatementURL'].values[0] #df is the dataframe created from csv
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            response = requests.get(url, timeout=10).content
            if is_response_html(response):
                print('The file is text '+OrgNm)
                text_response = clean_html_response(html_response=response)
                text_response = clean_html_response(html_response=text_response)
                save_text(text=text_response, filename=OrgNm)
            if is_response_pdf(response):
                print('The file is PDF '+OrgNm)# if pdf response
                save_pdf(pdf_response=response, filename=OrgNm)
            leftover_OrgNm.remove(OrgNm)
            time.sleep(2)
        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}")
            with lock:
                write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    with lock:
        write_pickle(Pickle_path,pkl_filename,leftover_OrgNm)
    print('Total Organization URLs processed '+ str(len(remaining_orgNm)-len(leftover_OrgNm)))
    print('Remaining number of organization data to extract '+str(len(leftover_OrgNm)))

In [100]:
import concurrent.futures
import requests
from threading import Thread
from threading import Lock

lock = Lock()
threads = [Thread(target=extract_texts_pdf2023, args=(ON_URLs_to_extract2023,'OrganisationName2023',lock))]

for thread in threads:
    thread.start()

Total Organization URLs to be processesd 7


1it [00:00,  1.52it/s]

Unexpected err=ConnectionError(MaxRetryError("HTTPSConnectionPool(host='www.frank-roberts.co.uk', port=443): Max retries exceeded with url: /serious-stuff/governance/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B8054B3850>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))")), type(err)=<class 'requests.exceptions.ConnectionError'>


2it [00:00,  2.25it/s]

Unexpected err=SSLError(MaxRetryError("HTTPSConnectionPool(host='www.ljmu.ac.uk', port=443): Max retries exceeded with url: /legal/modern-slavery-act-transparency-statement (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))")), type(err)=<class 'requests.exceptions.SSLError'>


...

Unexpected err=ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='www.mandmdirect.com', port=443): Read timed out. (read timeout=10)")), type(err)=<class 'requests.exceptions.ReadTimeout'>


6it [00:12,  2.73s/it]

Unexpected err=SSLError(MaxRetryError("HTTPSConnectionPool(host='www.ljmu.ac.uk', port=443): Max retries exceeded with url: /legal/modern-slavery-act-transparency-statement (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))")), type(err)=<class 'requests.exceptions.SSLError'>


7it [00:12,  1.80s/it]

Unexpected err=SSLError(MaxRetryError("HTTPSConnectionPool(host='www.ljmu.ac.uk', port=443): Max retries exceeded with url: /legal/modern-slavery-act-transparency-statement (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))")), type(err)=<class 'requests.exceptions.SSLError'>
Total Organization URLs processed 0
Remaining number of organization data to extract 7


In [ ]:
#